# Using Environ for Solvent and Electrolyte Effects on Slabs

## Tutorial Setup

Import all the Python modules needed in this tutorial. Most of these are very common in scientific computing, some are popular tools in atomistic simulations.

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.build import molecule, surface, bulk
from ase.visualize import view
from ase.calculators.espresso import Espresso

Setting up environment variables that are needed in order to submit simulations using Quantum Espresso through ASE

In [5]:
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x --environ -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

Some basic constants that may be useful later in the tutorials

In [4]:
eV2Ry = 13.605662285137 # energy conversion factor
eV2kcal_mol = 23.0609 # energy conversion factor
bohr2ang = 0.5291772 # length conversion factor
ang2bohr = 1./bohr2ang

## Local Functions and Classes Used for the Tutorial

In [9]:
def get_total_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the total energies from each scf calculation

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('!'):
            energies.append(float(line.split()[4]))
    return energies

def get_scf_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the energies from each scf step

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('total energy'):
            energies.append(float(line.split()[3]))
    return energies

def get_scf_accuracy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the energies from each scf step

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    accuracies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('estimated scf accuracy'):
            accuracies.append(float(line.split()[4]))
    return accuracies

def get_total_force(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the total force acting on the atoms from each scf calculation

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        forces = list of floats
    """
    forces=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('Total force'):
            forces.append(float(line.split()[3]))
    return forces


def get_bond_length(i,j,filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, and the indexes of two
    atoms in the simulation (using Python notation, starting from 0)
    extract the bond length for each scf step in the simulation

    Input Variables:
        i = index of first atom
        j = index of second atom
        filename = name of pw.x output file
    
    Output Variables:
        bonds = list of floats
    """
    bonds=[]
    ri = np.zeros(3)
    rj = np.zeros(3)
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('number of atoms'):
            nat = int(line.split()[4])
            break
    if i < 0 or i >= nat : 
        print('Error, index i must be >= 0 and < nat')
        return
    if j < 0 or j >= nat : 
        print('Error, index j must be >= 0 and < nat')
        return
    if i == j : 
        print('Error, indexes i and j must be different')
        return
    iat = -1
    for line in lines:
        if iat >= 0 and iat < nat : 
            if iat == i : ri = np.array([line.split()[1:4]],dtype=float)
            if iat == j : rj = np.array([line.split()[1:4]],dtype=float)
            iat += 1 
        if iat == nat : 
            bonds.append(np.sqrt(np.sum((ri-rj)**2)))
            iat = -1
        if line.strip().startswith('ATOMIC_POSITIONS'):
            iat = 0
    return bonds

In [7]:
class cubefile():
    """
    Class to read and manipulate cubefiles
    """

    def __init__(this,filename,delta=1.e-5):
        lines = [line for line in open(filename, 'r')]
        this.nat = int(lines[2].split()[0])
        this.origin = np.array(lines[2].split()[1:4],dtype=float)
        this.n = np.zeros(3,dtype=int)
        this.dgrid = np.zeros(3)
        for i in range(3):
            this.n[i] = int(lines[3+i].split()[0])
            this.dgrid[i] = float(lines[3+i].split()[i+1])
        this.atmnum = np.zeros(this.nat,dtype=int)
        this.atmchg = np.zeros(this.nat)
        this.atmpos = np.zeros((3,this.nat))
        for i in range(this.nat):
            this.atmnum[i] = int(lines[6+i].split()[0])
            this.atmchg[i] = float(lines[6+i].split()[1])
            this.atmpos[:,i] = np.array(lines[6+i].split()[2:5],dtype=float)
        this.ntot = this.n[0]*this.n[1]*this.n[2]
        this.data = np.zeros(this.ntot) 
        i = 0
        for line in lines[9:-1] : 
            this.data[i:i+6]=[ float(s) for s in line.split()]
            i += 6
        this.data[i:]=[ float(s) for s in line.split()]
        this.data = this.data.reshape((this.n[0],this.n[1],this.n[2]))
        this.r=np.zeros((3,this.n[0],this.n[1],this.n[2]))
        this.x=np.arange(0.,this.dgrid[0]*this.n[0]-delta,this.dgrid[0])+this.origin[0]
        this.y=np.arange(0.,this.dgrid[1]*this.n[1]-delta,this.dgrid[1])+this.origin[1]
        this.z=np.arange(0.,this.dgrid[2]*this.n[2]-delta,this.dgrid[2])+this.origin[2]
        this.r[0],this.r[1],this.r[2]=np.meshgrid(this.x,this.y,this.z,indexing='ij')

    def to_line(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            axis = this.r[0,:,icenter[1],icenter[2]]
            value = this.data[:,icenter[1],icenter[2]]
        elif axis == 1 :
            axis = this.r[1,icenter[0],:,icenter[2]]
            value = this.data[icenter[0],:,icenter[2]]
        elif axis == 2 :
            axis = this.r[2,icenter[0],icenter[1],:]
            value = this.data[icenter[0],icenter[1],:]
        return axis,value
    
    def to_line_planar_average(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            axis = this.r[0,:,icenter[1],icenter[2]]
            value = np.mean(this.data,(1,2))
        elif axis == 1 :
            axis = this.r[1,icenter[0],:,icenter[2]]
            value = np.mean(this.data,(0,2))
        elif axis == 2 :
            axis = this.r[2,icenter[0],icenter[1],:]
            value = np.mean(this.data,(0,1))
        return axis,value